In [1]:
import numpy as np
import statistics as st
import pandas as pd 

In [11]:
import pandas as pd
import statsmodels.api as sm
from scipy import stats

In [ ]:
import pandas as pd
import statsmodels.api as sm

# Tải dữ liệu từ các file CSV cho ngân hàng và GDP
df_TCB = pd.read_csv('TCB_adjusted_data.csv', encoding='ISO-8859-1')
df_VCB = pd.read_csv('VCB_adjusted_data.csv', encoding='ISO-8859-1')
df_MBB = pd.read_csv('MBB_adjusted_data.csv', encoding='ISO-8859-1')
df_BID = pd.read_csv('BID_adjusted_data.csv', encoding='ISO-8859-1')
df_GDP = pd.read_csv('gdp_quarterly_adjusted.csv', encoding='ISO-8859-1')

# Lọc dữ liệu theo ngày và chỉ lấy năm từ 2013 đến 2023
selected_columns = ['Date', 'roe', 'roa', 'operationProfit', 'postTaxProfit', 'interestMargin', 'operationIncome', 'creditGrowth']
df_TCB_filtered = df_TCB[(df_TCB['Date'] >= '2013-01-01') & (df_TCB['Date'] <= '2023-12-31')][selected_columns].add_prefix('TCB_')
df_VCB_filtered = df_VCB[(df_VCB['Date'] >= '2013-01-01') & (df_VCB['Date'] <= '2023-12-31')][selected_columns].add_prefix('VCB_')
df_MBB_filtered = df_MBB[(df_MBB['Date'] >= '2013-01-01') & (df_MBB['Date'] <= '2023-12-31')][selected_columns].add_prefix('MBB_')
df_BID_filtered = df_BID[(df_BID['Date'] >= '2013-01-01') & (df_BID['Date'] <= '2023-12-31')][selected_columns].add_prefix('BID_')
df_GDP_filtered = df_GDP[['Date', 'Quarterly_GDP']]
df_GDP_filtered['year'] = pd.to_datetime(df_GDP_filtered['Date']).dt.year
df_GDP_filtered = df_GDP_filtered[(df_GDP_filtered['year'] >= 2013) & (df_GDP_filtered['year'] <= 2023)]

# Đảm bảo cột 'year' tồn tại sau khi thêm tiền tố
df_TCB_filtered['year'] = pd.to_datetime(df_TCB_filtered['TCB_Date']).dt.year
df_VCB_filtered['year'] = pd.to_datetime(df_VCB_filtered['VCB_Date']).dt.year
df_MBB_filtered['year'] = pd.to_datetime(df_MBB_filtered['MBB_Date']).dt.year
df_BID_filtered['year'] = pd.to_datetime(df_BID_filtered['BID_Date']).dt.year

# Kết hợp dữ liệu ngân hàng và GDP theo từng năm
merged_data = df_GDP_filtered[['year', 'Quarterly_GDP']]
merged_data = merged_data.merge(df_TCB_filtered.drop(columns=['TCB_Date']), on='year', how='outer')
merged_data = merged_data.merge(df_VCB_filtered.drop(columns=['VCB_Date']), on='year', how='outer')
merged_data = merged_data.merge(df_MBB_filtered.drop(columns=['MBB_Date']), on='year', how='outer')
merged_data = merged_data.merge(df_BID_filtered.drop(columns=['BID_Date']), on='year', how='outer')

# Tính tăng trưởng GDP từ dữ liệu GDP hàng năm
merged_data['GDP_Growth'] = merged_data['Quarterly_GDP'].pct_change() * 100

# Loại bỏ các hàng có giá trị NaN để sử dụng cho hồi quy
cleaned_data = merged_data.dropna()

# Chọn các biến độc lập và biến phụ thuộc cho hồi quy tuyến tính
X = cleaned_data[['TCB_roe', 'TCB_roa', 'TCB_operationProfit', 'TCB_postTaxProfit', 'TCB_interestMargin', 'TCB_operationIncome', 'TCB_creditGrowth',
                  'VCB_roe', 'VCB_roa', 'VCB_operationProfit', 'VCB_postTaxProfit', 'VCB_interestMargin', 'VCB_operationIncome', 'VCB_creditGrowth',
                  'MBB_roe', 'MBB_roa', 'MBB_operationProfit', 'MBB_postTaxProfit', 'MBB_interestMargin', 'MBB_operationIncome', 'MBB_creditGrowth',
                  'BID_roe', 'BID_roa', 'BID_operationProfit', 'BID_postTaxProfit', 'BID_interestMargin', 'BID_operationIncome', 'BID_creditGrowth']]
y = cleaned_data['GDP_Growth']

# Thêm hằng số cho mô hình hồi quy
X = sm.add_constant(X)

# Xây dựng mô hình hồi quy tuyến tính đa biến
model = sm.OLS(y, X).fit()

# Xuất kết quả hồi quy
print(model.summary())

# Kết luận dựa trên các giá trị p-value của từng biến
significant_variables = model.pvalues[model.pvalues < 0.05].index.tolist()
if 'const' in significant_variables:
    significant_variables.remove('const')

if len(significant_variables) > 0:
    print("\nCác biến có tác động đáng kể đến tăng trưởng GDP:")
    for var in significant_variables:
        print(f"- {var}")
else:
    print("\nKhông có biến nào có tác động đáng kể đến tăng trưởng GDP.")


In [99]:
import pandas as pd
import statsmodels.api as sm

# Load data from CSV files for banks and GDP
df_TCB = pd.read_csv('TCB_adjusted_data.csv', encoding='ISO-8859-1')
df_VCB = pd.read_csv('VCB_adjusted_data.csv', encoding='ISO-8859-1')
df_MBB = pd.read_csv('MBB_adjusted_data.csv', encoding='ISO-8859-1')
df_BID = pd.read_csv('BID_adjusted_data.csv', encoding='ISO-8859-1')
df_GDP = pd.read_csv('gdp_quarterly_adjusted.csv', encoding='ISO-8859-1')

# Selected features with potential impact on GDP
selected_features = [
    'year', 'roe', 'roa', 'interestMargin', 'badDebtPercentage', 'creditGrowth', 
    'operationProfit', 'yearOperationProfitGrowth', 'postTaxProfit', 'provisionOnBadDebt', 
    'equityOnTotalAsset'
]

# Helper function to filter and rename columns for each bank
def filter_and_rename(data, bank_name):
    # Retain only columns present in the dataset
    available_features = [col for col in selected_features if col in data.columns]
    filtered_data = data[available_features].copy()
    filtered_data.columns = [f"{bank_name}_{col}" if col != 'year' else col for col in filtered_data.columns]
    return filtered_data

# Apply the filter function to each bank's dataset
vcb_filtered = filter_and_rename(df_VCB, 'VCB')
tcb_filtered = filter_and_rename(df_TCB, 'TCB')
mbb_filtered = filter_and_rename(df_MBB, 'MBB')
bid_filtered = filter_and_rename(df_BID, 'BID')

# Prepare GDP data by extracting the year from the Date column
df_GDP['year'] = pd.to_datetime(df_GDP['Date']).dt.year
gdp_q4 = df_GDP[(df_GDP['year'] >= 2013) & (df_GDP['year'] <= 2023)]

# Merge the bank data with GDP data on 'year'
merged_data = gdp_q4[['year', 'Quarterly_GDP']].merge(vcb_filtered, on='year') \
                                               .merge(tcb_filtered, on='year') \
                                               .merge(mbb_filtered, on='year') \
                                               .merge(bid_filtered, on='year')

# Define the independent variables (X) and dependent variable (y)
X = merged_data.drop(columns=['year', 'Quarterly_GDP'])  # Independent variables
y = merged_data['Quarterly_GDP']  # Dependent variable: Quarterly GDP growth

# Add a constant to the independent variables for the linear regression model
X = sm.add_constant(X)

# Fit the multivariate linear regression model
model = sm.OLS(y, X).fit()

# Print the model summary to show coefficients and p-values
print(model.summary())

# Extract and print significant features based on p-value < 0.05
significant_features = model.pvalues[model.pvalues < 0.05].index.tolist()
print("Significant features with p-value < 0.05:", significant_features)


                            OLS Regression Results                            
Dep. Variable:          Quarterly_GDP   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                 2.854e+04
Date:                Wed, 06 Nov 2024   Prob (F-statistic):               0.00
Time:                        19:25:07   Log-Likelihood:            -1.3066e+05
No. Observations:               10752   AIC:                         2.614e+05
Df Residuals:                   10723   BIC:                         2.616e+05
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   1.45